In [159]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import metrics
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization
from keras.layers import LSTM
from keras.layers import GRU
from keras import regularizers
import keras
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import shuffle
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
import seaborn as sn
import itertools  
from sklearn.utils.class_weight import compute_class_weight
from imblearn.over_sampling import RandomOverSampler
from collections import Counter
import random

In [95]:
def print_cm(cm, labels, hide_zeroes=False, hide_diagonal=False, hide_threshold=None):
    """pretty print for confusion matrixes"""
    columnwidth = max([len(x) for x in labels] + [5])  # 5 is value length
    empty_cell = " " * columnwidth
    # Print header
    print("    " + empty_cell, end=" ")
    for label in labels:
        print("%{0}s".format(columnwidth) % label, end=" ")
    print()
    # Print rows
    for i, label1 in enumerate(labels):
        print("    %{0}s".format(columnwidth) % label1, end=" ")
        for j in range(len(labels)):
            cell = "%{0}.1f".format(columnwidth) % cm[i, j]
            if hide_zeroes:
                cell = cell if float(cm[i, j]) != 0 else empty_cell
            if hide_diagonal:
                cell = cell if i != j else empty_cell
            if hide_threshold:
                cell = cell if cm[i, j] > hide_threshold else empty_cell
            print(cell, end=" ")
        print()

In [96]:
#Loading the data with State Avg column

cdc = pd.read_csv('/Users/sparshagarwal/Desktop/NCSA/Dataframes/WNV_challenge_neighCountyAvg.csv')

#Changes/Cleaning that needs to be done in data as mentioned by the cdc manual

cdc['county'] = np.where(cdc['county']=='Bedford/Bedford City', 'Bedford', cdc['county'])
cdc['fips'] = np.where(cdc['fips']=='51019/51515', '51019', cdc['fips'])
cdc['location'] = np.where(cdc['location']=='Virginia-Bedford/Bedford City', 'Virginia-Bedford', cdc['location'])

cdc['county'] = np.where(cdc['county']=='Oglala Lakota/Shannon', 'Oglala Lakota', cdc['county'])
cdc['fips'] = np.where(cdc['fips']=='46102/46113', '46102', cdc['fips'])
cdc['location'] = np.where(cdc['location']=='South Dakota-Oglala Lakota/Shannon', 'South Dakota-Oglala Lakota', cdc['location'])


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pandas/core/ops/__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


In [97]:
#converting fips from object/string to int datatype

cdc['fips']=cdc['fips'].apply(pd.to_numeric)

In [98]:
bin_dummy={1:[], 2:[], 3:[], 4:[], 5:[], 6:[], 7:[], 8:[], 9:[], 10:[], 11:[], 12:[], 13:[], 14:[], 15:[]}

for i in range(len(cdc)):
    cl=cdc.iloc[i]["bin"]
    for j in bin_dummy:
        bin_dummy[j].append(0)
    bin_dummy[cl][-1]=1
    
for i in bin_dummy:
    cdc[i]=bin_dummy[i]

In [99]:
cdc.head()

,fips,county,state,location,year,count,bin,neigh_death_count,num_of_neighbor,neighborCountyAvg,...,6,7,8,9,10,11,12,13,14,15
0,1001,Autauga,Alabama,Alabama-Autauga,2000,0,1,0,5,0.0,...,0,0,0,0,0,0,0,0,0,0
1,1001,Autauga,Alabama,Alabama-Autauga,2001,0,1,0,5,0.0,...,0,0,0,0,0,0,0,0,0,0
2,1001,Autauga,Alabama,Alabama-Autauga,2002,1,2,8,5,1.6,...,0,0,0,0,0,0,0,0,0,0
3,1001,Autauga,Alabama,Alabama-Autauga,2003,0,1,2,5,0.4,...,0,0,0,0,0,0,0,0,0,0
4,1001,Autauga,Alabama,Alabama-Autauga,2004,0,1,0,5,0.0,...,0,0,0,0,0,0,0,0,0,0


In [146]:
years=range(2000,2019)
num_years=5
features=["count","neighborCountyAvg"]


min_year=years[0]+num_years+1
max_year=years[-1]

In [147]:
add_features=features.copy()
add_features.extend(["year","location"])
add_features.extend(i for i in range(1,16))

In [148]:
temp_df=cdc[add_features]

In [149]:
#Creating dataframe for training data
temporal_df=pd.DataFrame()                   #The final dataframe
for year in range(max_year, min_year-1, -1): # loop from 2018 to 2002 with step of -1 in each round
    col="A"
    yearly_df=pd.DataFrame()                 # created yearly_df empty dataframe
    target=temp_df[temp_df["year"]==year]    # created new df with name as target and calling on per year basis
    classes=[i for i in range(1,16)]         # list of classes
    target=target[classes]                   # target df updated with only that year classes
    for prior_year in range(year-2,year-num_years-2,-1):  # loop from 2016 to 2014 with step of -1 in each round
        df_prior_year=temp_df[temp_df["year"]==prior_year]
        for feature in features:
            feat_values=list(df_prior_year[feature])
            yearly_df[col]=feat_values
            col=chr(ord(col)+1)              
    yearly_df=pd.concat([yearly_df.reset_index(),target.reset_index()], axis=1).drop(["index"], axis=1)
    temporal_df=temporal_df.append(yearly_df)

In [150]:
#Creating dataframe for testing data
year=2020
col="A"
temporal_df_test=pd.DataFrame()
for prior_year in range(year-2,year-num_years-2,-1):
    df_prior_year=temp_df[temp_df["year"]==prior_year]
    for feature in features:
        feat_values=list(df_prior_year[feature])
        temporal_df_test[col]=feat_values
        col=chr(ord(col)+1)

In [151]:
temporal_df.head()

,A,B,C,D,E,F,G,H,I,J,...,6,7,8,9,10,11,12,13,14,15
0,1,0.000000,0,0.2,0,0.000000,0,0.000000,0,1.600000,...,0,0,0,0,0,0,0,0,0,0
1,2,0.666667,0,1.0,0,0.333333,1,0.000000,4,3.000000,...,0,0,0,0,0,0,0,0,0,0
2,0,0.000000,0,0.0,0,0.000000,0,0.000000,0,0.000000,...,0,0,0,0,0,0,0,0,0,0
3,0,0.500000,1,0.0,0,0.000000,0,0.166667,0,1.166667,...,0,0,0,0,0,0,0,0,0,0
4,0,0.333333,0,0.0,0,0.000000,0,0.000000,0,0.833333,...,0,0,0,0,0,0,0,0,0,0


In [152]:
#Checking frequency of instances for each class
dict_freq={}
for i in range(1,16):
    dict_freq[i]=len(temporal_df[temporal_df[i]==1])
dict_freq

{1: 35220,
 2: 4654,
 3: 312,
 4: 88,
 5: 39,
 6: 28,
 7: 13,
 8: 8,
 9: 5,
 10: 5,
 11: 5,
 12: 15,
 13: 7,
 14: 3,
 15: 2}

In [153]:
X_pre = temporal_df.iloc[:, 0:(num_years*len(features))]
Y_pre = temporal_df.iloc[:, (num_years*len(features)):]

In [169]:
X=X_pre.values
Y=Y_pre.values

In [154]:
# #Balancing the data w.r.t. class labels
# ros = RandomOverSampler()
# X, Y = ros.fit_resample(X_pre.values,Y_pre.values)

In [155]:
#Shows all the class labels are now equally represented
y=np.argmax(Y, axis=1)
np.bincount(y)

array([35220, 35220, 35220, 35220, 35220, 35220, 35220, 35220, 35220,
       35220, 35220, 35220, 35220, 35220, 35220])

In [156]:
# # Calculating classs weights based on their frequencies
# y=np.argmax(temporal_df.iloc[:,-15:].values, axis=1)
# class_weights=compute_class_weight("balanced", [i for i in range(15)], y)

## LSTM Implementation for classification

In [170]:
def create_network():
    network = Sequential()
    network.add(BatchNormalization(input_shape=(num_years, len(features))))
    network.add(Dense(4, activation="tanh"))
    network.add(GRU(15, dropout = 0.2, recurrent_dropout = 0.2, activation="tanh"))
    network.add(Dense(15, activation="softmax"))
    network.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return network

In [171]:
#Cross-validation setup
acc_train=[]
acc_val=[]
kf = KFold(n_splits=5)
for train_index, val_index in kf.split(X):   
#     X_train=temporal_df.iloc[train_index,0:(num_years*len(features))]
#     Y_train=temporal_df.iloc[train_index,(num_years*len(features)):]
#     X_val=temporal_df.iloc[val_index,0:(num_years*len(features))]
#     Y_val=temporal_df.iloc[val_index,(num_years*len(features)):]
    X_train=X[train_index]
    Y_train=Y[train_index]
    X_val=X[val_index]
    Y_val=Y[val_index]
    
    #Transforming input variables into LSTM input format
    X_train = X_train.reshape(X_train.shape[0], num_years, len(features))
    X_val = X_val.reshape(X_val.shape[0], num_years, len(features))
    Y_train=Y_train
    Y_val=Y_val
    
    #Creating model
    model=create_network()
    Hist=model.fit(X_train, Y_train, epochs=10, validation_data=(X_val, Y_val), verbose=2, class_weight=None)
    
    #Final epoch accuraies for training and validation dataset
    acc_train.append(Hist.history["accuracy"][-1])
    acc_val.append(Hist.history["val_accuracy"][-1])
    
print("Training accuracy:" + str(np.mean(acc_train)))
print("Validation accuracy:" + str(np.mean(acc_val)))
    

Train on 32323 samples, validate on 8081 samples
Epoch 1/10
 - 5s - loss: 0.5673 - accuracy: 0.8628 - val_loss: 0.4701 - val_accuracy: 0.8349
Epoch 2/10
 - 4s - loss: 0.3821 - accuracy: 0.8814 - val_loss: 0.4558 - val_accuracy: 0.8349
Epoch 3/10
 - 4s - loss: 0.3747 - accuracy: 0.8825 - val_loss: 0.4516 - val_accuracy: 0.8329
Epoch 4/10
 - 4s - loss: 0.3693 - accuracy: 0.8823 - val_loss: 0.4410 - val_accuracy: 0.8360
Epoch 5/10
 - 4s - loss: 0.3689 - accuracy: 0.8825 - val_loss: 0.4401 - val_accuracy: 0.8337
Epoch 6/10
 - 4s - loss: 0.3665 - accuracy: 0.8828 - val_loss: 0.4403 - val_accuracy: 0.8322
Epoch 7/10
 - 4s - loss: 0.3667 - accuracy: 0.8827 - val_loss: 0.4407 - val_accuracy: 0.8312
Epoch 8/10
 - 4s - loss: 0.3649 - accuracy: 0.8834 - val_loss: 0.4435 - val_accuracy: 0.8316
Epoch 9/10
 - 4s - loss: 0.3634 - accuracy: 0.8832 - val_loss: 0.4374 - val_accuracy: 0.8317
Epoch 10/10
 - 4s - loss: 0.3622 - accuracy: 0.8835 - val_loss: 0.4369 - val_accuracy: 0.8328
Train on 32323 sampl

In [172]:
# Dataset while using test_train split for final prediction

#Transforming input variables into LSTM input format

X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.20)

X_train = X_train.reshape(X_train.shape[0], num_years, len(features))
X_val = X_val.reshape(X_val.shape[0], num_years, len(features))
Y_train=Y_train
Y_val=Y_val

X_test=temporal_df_test.iloc[:, 0:(num_years*len(features))]
X_test = X_test.values.reshape(X_test.shape[0], num_years, len(features))

In [173]:
model=create_network()

In [174]:
Hist=model.fit(X_train, Y_train, nb_epoch=10, validation_data=(X_val, Y_val), verbose=2, class_weight=None)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.


Train on 32323 samples, validate on 8081 samples
Epoch 1/10
 - 5s - loss: 0.5812 - accuracy: 0.8495 - val_loss: 0.3991 - val_accuracy: 0.8739
Epoch 2/10
 - 4s - loss: 0.3961 - accuracy: 0.8726 - val_loss: 0.3858 - val_accuracy: 0.8746
Epoch 3/10
 - 4s - loss: 0.3905 - accuracy: 0.8722 - val_loss: 0.3832 - val_accuracy: 0.8735
Epoch 4/10
 - 4s - loss: 0.3851 - accuracy: 0.8735 - val_loss: 0.3826 - val_accuracy: 0.8743
Epoch 5/10
 - 4s - loss: 0.3847 - accuracy: 0.8738 - val_loss: 0.3832 - val_accuracy: 0.8733
Epoch 6/10
 - 4s - loss: 0.3817 - accuracy: 0.8733 - val_loss: 0.3801 - val_accuracy: 0.8741
Epoch 7/10
 - 4s - loss: 0.3816 - accuracy: 0.8742 - val_loss: 0.3927 - val_accuracy: 0.8743
Epoch 8/10
 - 4s - loss: 0.3820 - accuracy: 0.8740 - val_loss: 0.3839 - val_accuracy: 0.8745
Epoch 9/10
 - 4s - loss: 0.3789 - accuracy: 0.8738 - val_loss: 0.3976 - val_accuracy: 0.8743
Epoch 10/10
 - 4s - loss: 0.3788 - accuracy: 0.8738 - val_loss: 0.3957 - val_accuracy: 0.8748


In [175]:
#Predicting value for train, val, and test datasets
pred_train=model.predict(X_train)
pred_val=model.predict(X_val)
pred_test=model.predict(X_test)

In [176]:
#Converting probabilities to class labels
pred_train_class=np.argmax(pred_train, axis=1)+1
pred_train_class=list(map(lambda x: str(x), pred_train_class))
pred_val_class=np.argmax(pred_val, axis=1)+1
pred_val_class=list(map(lambda x: str(x), pred_val_class))
pred_test_class=np.argmax(pred_test, axis=1)+1

In [177]:
true_train_class=np.argmax(Y_train, axis=1)+1
true_train_class=list(map(lambda x: str(x), true_train_class))
true_val_class=np.argmax(Y_val, axis=1)+1
true_val_class=list(map(lambda x: str(x), true_val_class))

In [178]:
labels = [str(i) for i in range(1,16)]
cm_train = confusion_matrix(true_train_class, pred_train_class , labels)
#cm_train=cm_train.astype('float') / cm_train.sum(axis=1)[:, np.newaxis] #For normalizing
cm_val = confusion_matrix(true_val_class, pred_val_class , labels)
#cm_val=cm_val.astype('float') / cm_val.sum(axis=1)[:, np.newaxis] #For normalizing

In [179]:
print_cm(cm_train, labels)

              1     2     3     4     5     6     7     8     9    10    11    12    13    14    15 
        1 28089.0  85.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0 
        2 3542.0 179.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0 
        3 181.0  71.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0 
        4  36.0  35.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0 
        5  16.0  15.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0 
        6  13.0   9.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0 
        7   7.0   5.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0 
        8   5.0   2.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0 
        9   0.0   2.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0 

In [180]:
print_cm(cm_val, labels)

              1     2     3     4     5     6     7     8     9    10    11    12    13    14    15 
        1 7025.0  21.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0 
        2 889.0  44.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0 
        3  45.0  15.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0 
        4   8.0   9.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0 
        5   7.0   1.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0 
        6   2.0   4.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0 
        7   1.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0 
        8   0.0   1.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0 
        9   3.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   0.0   

In [181]:
#Evaluation metrics for valdation dataset
print(metrics.classification_report(true_val_class, pred_val_class))

              precision    recall  f1-score   support

           1       0.88      1.00      0.93      7046
          10       0.00      0.00      0.00         2
          11       0.00      0.00      0.00         1
          12       0.00      0.00      0.00         1
          13       0.00      0.00      0.00         1
          14       0.00      0.00      0.00         1
           2       0.44      0.05      0.09       933
           3       0.00      0.00      0.00        60
           4       0.00      0.00      0.00        17
           5       0.00      0.00      0.00         8
           6       0.00      0.00      0.00         6
           7       0.00      0.00      0.00         1
           8       0.00      0.00      0.00         1
           9       0.00      0.00      0.00         3

    accuracy                           0.87      8081
   macro avg       0.09      0.07      0.07      8081
weighted avg       0.82      0.87      0.83      8081



/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## LSTM Implementation for regression

In [352]:
def create_network():
    network = Sequential()
    network.add(BatchNormalization(input_shape=(num_years, len(features))))
    network.add(Dense(4, activation="tanh"))
    network.add(LSTM(15, dropout = 0.2, recurrent_dropout = 0.2, activation="tanh"))
    network.add(Dense(1, activation="tanh"))
    network.compile(loss='mse', optimizer='adam', metrics=['accuracy'])
    
    return network

In [350]:
#Cross-validation setup
acc_train=[]
acc_val=[]
kf = KFold(n_splits=2)
for train_index, val_index in kf.split(temporal_df):   
    X_train=temporal_df.iloc[train_index,0:(num_years*len(features))]
    Y_train=temporal_df.iloc[train_index,(num_years*len(features)):]
    X_val=temporal_df.iloc[val_index,0:(num_years*len(features))]
    Y_val=temporal_df.iloc[val_index,(num_years*len(features)):]
    
    #Transforming input variables into LSTM input format
    X_train = X_train.values.reshape(X_train.shape[0], num_years, len(features))
    X_val = X_val.values.reshape(X_val.shape[0], num_years, len(features))
    Y_train=Y_train.values
    Y_val=Y_val.values
    
    #Creating model
    model=create_network()
    Hist=model.fit(X_train, Y_train, epochs=1, validation_data=(X_val, Y_val), verbose=2)
    
    #Final epoch accuracies for training and validation dataset
    acc_train.append(Hist.history["accuracy"][-1])
    acc_val.append(Hist.history["val_accuracy"][-1])
    
print("Training accuracy:" + str(np.mean(acc_train)))
print("Validation accuracy:" + str(np.mean(acc_val)))
    

Train on 12432 samples, validate on 12432 samples
Epoch 1/1
 - 3s - loss: 0.7554 - accuracy: 0.8436 - val_loss: 0.4844 - val_accuracy: 0.8439
Train on 12432 samples, validate on 12432 samples
Epoch 1/1
 - 3s - loss: 0.7612 - accuracy: 0.8254 - val_loss: 0.4337 - val_accuracy: 0.8642
Training accuracy:0.8344997
Validation accuracy:0.8540459871292114


array([7.82e-02, 5.04e-01, 7.47e+00, 2.81e+01, 6.63e+01, 8.72e+01,
       2.37e+02, 2.37e+02, 8.29e+02, 4.14e+02, 5.53e+02, 1.51e+02,
       2.76e+02, 5.53e+02, 8.29e+02])

In [61]:
estimator = KerasClassifier(build_fn=create_network, epochs=200, batch_size=64, verbose=2)

kfold = KFold(n_splits=10, shuffle=True)
results = cross_val_score(estimator, X_train, Y_train, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

# network.fit(X_train, Y_train, epochs=100, validation_data=(X_test, Y_test), verbose = 2)

Epoch 1/200
 - 2s - loss: 1.3761 - accuracy: 0.8402
Epoch 2/200
 - 1s - loss: 0.5475 - accuracy: 0.8655
Epoch 3/200
 - 1s - loss: 0.4853 - accuracy: 0.8655
Epoch 4/200
 - 1s - loss: 0.4663 - accuracy: 0.8655
Epoch 5/200
 - 1s - loss: 0.4594 - accuracy: 0.8655
Epoch 6/200
 - 1s - loss: 0.4565 - accuracy: 0.8655
Epoch 7/200
 - 1s - loss: 0.4539 - accuracy: 0.8655
Epoch 8/200
 - 1s - loss: 0.4541 - accuracy: 0.8651
Epoch 9/200
 - 1s - loss: 0.4518 - accuracy: 0.8644
Epoch 10/200
 - 1s - loss: 0.4490 - accuracy: 0.8652
Epoch 11/200
 - 1s - loss: 0.4493 - accuracy: 0.8652
Epoch 12/200
 - 1s - loss: 0.4468 - accuracy: 0.8648
Epoch 13/200
 - 1s - loss: 0.4440 - accuracy: 0.8658
Epoch 14/200
 - 1s - loss: 0.4433 - accuracy: 0.8657
Epoch 15/200
 - 1s - loss: 0.4426 - accuracy: 0.8649
Epoch 16/200
 - 1s - loss: 0.4406 - accuracy: 0.8669
Epoch 17/200
 - 1s - loss: 0.4400 - accuracy: 0.8659
Epoch 18/200
 - 1s - loss: 0.4394 - accuracy: 0.8652
Epoch 19/200
 - 1s - loss: 0.4374 - accuracy: 0.8659
Ep

KeyboardInterrupt: 

## Dense Layer Implementation

In [60]:
def create_network():
    network = Sequential()
    network.add(BatchNormalization())
    network.add(Dense(3000, activation="relu", input_shape=(39782,)))
    network.add(Dense(1500, activation="relu"))
    network.add(Dense(300, activation="relu"))
    network.add(Dense(15, activation="softmax"))

    network.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return network

In [63]:
X = temporal_df.iloc[:, 0:4].values
Y = temporal_df.iloc[:, 4:].values

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.20)

estimator = KerasClassifier(build_fn=create_network, epochs=8, batch_size=64, verbose=2)

kfold = KFold(n_splits=10, shuffle=True)
results = cross_val_score(estimator, X_train, Y_train, cv=kfold)

Epoch 1/8
 - 30s - loss: 0.4627 - accuracy: 0.8632
Epoch 2/8
 - 30s - loss: 0.4319 - accuracy: 0.8668
Epoch 3/8
 - 30s - loss: 0.4278 - accuracy: 0.8664
Epoch 4/8
 - 30s - loss: 0.4261 - accuracy: 0.8661
Epoch 5/8
 - 30s - loss: 0.4256 - accuracy: 0.8661
Epoch 6/8
 - 31s - loss: 0.4247 - accuracy: 0.8668
Epoch 7/8
 - 32s - loss: 0.4238 - accuracy: 0.8667
Epoch 8/8
 - 30s - loss: 0.4224 - accuracy: 0.8676
Epoch 1/8
 - 33s - loss: 0.4635 - accuracy: 0.8644
Epoch 2/8
 - 31s - loss: 0.4290 - accuracy: 0.8678
Epoch 3/8
 - 30s - loss: 0.4253 - accuracy: 0.8676
Epoch 4/8
 - 31s - loss: 0.4237 - accuracy: 0.8675
Epoch 5/8
 - 31s - loss: 0.4218 - accuracy: 0.8678
Epoch 6/8
 - 30s - loss: 0.4206 - accuracy: 0.8689
Epoch 7/8
 - 30s - loss: 0.4200 - accuracy: 0.8682
Epoch 8/8
 - 30s - loss: 0.4201 - accuracy: 0.8687
Epoch 1/8
 - 31s - loss: 0.4646 - accuracy: 0.8620
Epoch 2/8
 - 29s - loss: 0.4312 - accuracy: 0.8659
Epoch 3/8
 - 29s - loss: 0.4283 - accuracy: 0.8667
Epoch 4/8
 - 30s - loss: 0.4257

In [64]:
# This is a reasonable estimation of the performance of the model on unseen data.

print('Accuracy: %.2f%% (%.3f%%)' % (results.mean()*100, results.std()*100))

Accuracy: 86.56% (0.686%)


In [66]:
network = create_network()
network.fit(X_train, Y_train, epochs=8, validation_data=(X_test, Y_test), verbose = 2)

Train on 39782 samples, validate on 9946 samples
Epoch 1/8
 - 74s - loss: 0.4495 - accuracy: 0.8642 - val_loss: 0.4511 - val_accuracy: 0.8674
Epoch 2/8
 - 69s - loss: 0.4313 - accuracy: 0.8647 - val_loss: 0.4353 - val_accuracy: 0.8682
Epoch 3/8
 - 69s - loss: 0.4274 - accuracy: 0.8667 - val_loss: 0.4306 - val_accuracy: 0.8678
Epoch 4/8
 - 72s - loss: 0.4258 - accuracy: 0.8660 - val_loss: 0.4265 - val_accuracy: 0.8678
Epoch 5/8
 - 70s - loss: 0.4262 - accuracy: 0.8662 - val_loss: 0.4274 - val_accuracy: 0.8688
Epoch 6/8
 - 71s - loss: 0.4253 - accuracy: 0.8664 - val_loss: 0.4248 - val_accuracy: 0.8680
Epoch 7/8
 - 71s - loss: 0.4231 - accuracy: 0.8665 - val_loss: 0.4242 - val_accuracy: 0.8680
Epoch 8/8
 - 70s - loss: 0.4229 - accuracy: 0.8670 - val_loss: 0.4302 - val_accuracy: 0.8668


In [77]:
results2 = network.evaluate(X_test, Y_test)

9946/9946 [==============================] - 2s 237us/step


In [85]:
print('Accuracy: %.2f%% \nLoss: %.3f' % (results2[1]*100, results2[0]))

Accuracy: 86.68% 
Loss: 0.430


In [129]:
Y_pred_test = network.predict(X_test)

In [137]:
Y_pred_test = pd.DataFrame(Y_pred_test)
Y_pred_test = Y_pred_test.T
for i in Y_pred_test.columns.tolist():
    Y_pred_test[i] = np.where(Y_pred_test[i] == max(Y_pred_test[i]), 1, 0)

Y_pred_test = Y_pred_test.T

In [138]:
Y_pred_test.head(10)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [139]:
Y_test = pd.DataFrame(Y_test)
Y_test.head(10)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
6,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
8,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
